## Imports:

In [8]:
import wrangle
import explore

## Acquire:

***
#### Web-Scraping GitHub with Beautiful Soup
***

***1. Identify HTML target feature using Beautiful Soup***

`Repo README.md text` - soup.select('article', class_="markdown-body entry-content container-lg")[0].text

`Repo primary language` - soup.select('li.d-inline:nth-child(1) > a:nth-child(1)')[0].text

***2. Code extraction functions used to create corpus***

`def get_soup()`
This function takes in a URL, parses the HTML and returns a BeautifulSoup object

`def gethub_geology_urls()`
This function returns a list of URLs that can be read in the get_readme_articles() function

`def get_geo_results()`
This function scrapes and returns a list of repo URLs from search results

#### Explanation:

Using the python library, Beautiful Soup, the README.md text of 507 GitHub repositories were scraped. I focused my search exclusively on repositories identifed by the keyword'Geology'. The results of this scrape were saved in a json file and were used as our research corpus.

## Prepare:

In [2]:
def prep_data(df, column):
    df['content_clean'] = df[column].apply(basic_clean)\
                                    .apply(tokenize)\
                                    .apply(lemmatize)\
                                    .apply(remove_stopwords)\
                                    .apply(remove_extra_words)

    # add a column with a list of words
    words = [re.sub(r'([^a-z0-9\s]|\s.\s)', '', doc).split() for doc in df.content_clean]

    # column name will be words, and the column will contain lists of the words in each doc
    df = pd.concat([df, pd.DataFrame({'words': words})], axis=1)

    # add column with number of words in readme content
    df['doc_length'] = [len(wordlist) for wordlist in df.words]
    
    # Adds column with bigrams and trigrams
    df['bigrams'] =  df['text_filtered'].apply(lambda row: list(nltk.bigrams(row.split(' '))))
    df['trigrams'] =  df['text_filtered'].apply(lambda row: list(nltk.trigrams(row.split(' '))))
    
    # Specify dataframe content
    df = df[['language','content','content_clean','doc_length','words','bigrams','trigrams']]
    return df


#### Explanation:

`basic_clean( ):` This function changes the text to lowercase, removes special characters, encodes to ascii and recode to utf-8 to remove numbers from the text, and finally replaces anything that is not a letter, number, whitespace, or a single quote with an empty string. 

`tokenize( ):` This function takes in a string and returns that string broken up in to a list of individual words.

`lemmatize( ):` This function takes in string for and returns a string with words simplified to dictionary roots.

`remove_stopwords():` This function is a spealized cleaning tool from the Gensim library that cleans and parses 
                      text for topic modeling and removes more common words then the traditional nltk library.

#### Natural Language ToolKit (NLTK)

-NLTK library has 179 words in the stopword collection.

Note: All the words in the default library’s stopword list are in lower case, that means documents/sentences words also must be lower case. Otherwise, stopword not got removed from your data.

In [3]:
from nltk.corpus import stopwords
nltk_stopwords = set(stopwords.words('english'))
text = f"The first time I saw Catherine she was wearing a vivid crimson dress and was nervously " \
f"leafing through a magazine in my waiting room."
text_without_stopword = [word for word in text.split() if word.lower() not in nltk_stopwords]
print(f"Original Text : {text}")
print(f"Text without stopwords : {' '.join(text_without_stopword)}")
print(f"Total count of stopwords in NLTK is {len(nltk_stopwords)}")

Original Text : The first time I saw Catherine she was wearing a vivid crimson dress and was nervously leafing through a magazine in my waiting room.
Text without stopwords : first time saw Catherine wearing vivid crimson dress nervously leafing magazine waiting room.
Total count of stopwords in NLTK is 179


#### Gensim Library

-Gensim has 337 words in their stopwords collection

Note: 

In [4]:
from gensim.parsing.preprocessing import remove_stopwords
import gensim

gensim_stopwords = gensim.parsing.preprocessing.STOPWORDS

text = f"The first time I saw Catherine she was wearing a vivid crimson dress and was nervously " \
       f"leafing through a magazine in my waiting room."

print(f"Original Text : {text}")
print(f"Text without stopwords : {remove_stopwords(text.lower())}")
print(f"Total count of stopwords in Ginsim is {len(gensim_stopwords)}")

Original Text : The first time I saw Catherine she was wearing a vivid crimson dress and was nervously leafing through a magazine in my waiting room.
Text without stopwords : time saw catherine wearing vivid crimson dress nervously leafing magazine waiting room.
Total count of stopwords in Ginsim is 337


## Data Exploration: (Topic Modeling)

1. Create LDA Object
2. Fit to content
3. Use LDA to predict (n) Topics
4. Explore topics for correlations
5. Repeat and fine tune model parameters

#### Call in Clean Data Frame

In [5]:
df = wrangle.get_geo_results(cached=True)
df = wrangle.prep_data(df, 'content')
df.head()

,language,content,content_clean,doc_length,words,bigrams,trigrams
0,Python,Map Merger tool - tested using ArcMap 10.7\nWr...,map merger tool tested arcmap written ryan cro...,38,"[map, merger, tool, tested, arcmap, written, r...","[(map, merger), (merger, tool), (tool, tested)...","[(map, merger, tool), (merger, tool, tested), ..."
1,Jupyter Notebook,wellio.js\nJavaScript for converting well-log ...,welliojs javascript converting welllog standar...,1053,"[welliojs, javascript, converting, welllog, st...","[(welliojs, javascript), (javascript, converti...","[(welliojs, javascript, converting), (javascri..."
2,Python,geomodel-2-3dweb\n\nGenerates 3D web versions ...,geomodeldweb generates d web version geologica...,152,"[geomodeldweb, generatesweb, version, geologic...","[(geomodeldweb, generates), (generates, d), (d...","[(geomodeldweb, generates, d), (generates, d, ..."
3,JavaScript,GeoFeature\nGeological features of the Quanfoc...,geofeature geological feature quanfock hill de...,166,"[geofeature, geological, feature, quanfock, hi...","[(geofeature, geological), (geological, featur...","[(geofeature, geological, feature), (geologica..."
4,JavaScript,U.S. Geological Survey Best Practices\nThis re...,u geological survey best practice repository h...,13,"[u, geological, survey, best, practice, reposi...","[(u, geological), (geological, survey), (surve...","[(u, geological, survey), (geological, survey,..."


### Topic Modeling: Step One (Applying LDA)

Before we can apply LDA, we need to create vocabulary of all the words in our data. Remember from the previous article, we could do so with the help of a count vectorizer.

sklearn.feature_extraction.text module to create a document-term matrix. We specify to only include those words that appear in less than 80% of the document and appear in at least 2 documents. We also remove all the stop words as they do not really contribute to topic modeling.


In [6]:
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.decomposition import LatentDirichletAllocation
# import random

# def nlp_topic_modeling(df, max_df, min_df, n_components):
#     count_vect = CountVectorizer(max_df=max_df, min_df=min_df, stop_words='english')
#     doc_term_matrix = count_vect.fit_transform(df['content_clean'].values.astype('U'))
#     LDA = LatentDirichletAllocation(n_components=n_components, random_state=123)
#     LDA.fit(doc_term_matrix)
#     for i,topic in enumerate(LDA.components_):
#         print(f'Top {n_components} words for topic #{i}:')
#         print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
#         print('\n')
#     topic_values = LDA.transform(doc_term_matrix)
#     df['Topic'] = topic_values.argmax(axis=1)
#     return df

In [13]:
df = explore.nlp_topic_modeling(df, max_df = 0.8 , min_df = 2 , n_components = 5)
df.head()

Top 5 words for topic #0:
['geological', 'usgs', 'text', 'report', 'set', 'project', 'earthquake', 'file', 'map', 'data']


Top 5 words for topic #1:
['example', 'code', 'geological', 'geology', 'version', 'python', 'run', 'model', 'data', 'file']


Top 5 words for topic #2:
['line', 'package', 'notebook', 'powerlaw', 'exercise', 'deposit', 'pip', 'conda', 'python', 'install']


Top 5 words for topic #3:
['code', 'app', 'project', 'npm', 'test', 'build', 'use', 'run', 'file', 'enabled']


Top 5 words for topic #4:
['earthquake', 'usgs', 'file', 'build', 'use', 'geological', 'project', 'app', 'android', 'license']




,language,content,content_clean,doc_length,words,bigrams,trigrams,Topic
0,Python,Map Merger tool - tested using ArcMap 10.7\nWr...,map merger tool tested arcmap written ryan cro...,38,"[map, merger, tool, tested, arcmap, written, r...","[(map, merger), (merger, tool), (tool, tested)...","[(map, merger, tool), (merger, tool, tested), ...",4
1,Jupyter Notebook,wellio.js\nJavaScript for converting well-log ...,welliojs javascript converting welllog standar...,1053,"[welliojs, javascript, converting, welllog, st...","[(welliojs, javascript), (javascript, converti...","[(welliojs, javascript, converting), (javascri...",0
2,Python,geomodel-2-3dweb\n\nGenerates 3D web versions ...,geomodeldweb generates d web version geologica...,152,"[geomodeldweb, generatesweb, version, geologic...","[(geomodeldweb, generates), (generates, d), (d...","[(geomodeldweb, generates, d), (generates, d, ...",4
3,JavaScript,GeoFeature\nGeological features of the Quanfoc...,geofeature geological feature quanfock hill de...,166,"[geofeature, geological, feature, quanfock, hi...","[(geofeature, geological), (geological, featur...","[(geofeature, geological, feature), (geologica...",2
4,JavaScript,U.S. Geological Survey Best Practices\nThis re...,u geological survey best practice repository h...,13,"[u, geological, survey, best, practice, reposi...","[(u, geological), (geological, survey), (surve...","[(u, geological, survey), (geological, survey,...",0


In [14]:
df.Topic.value_counts()

0    175
1    144
3     72
4     71
2     45
Name: Topic, dtype: int64